### Install dependencies

In [1]:
from jupyter_dash import JupyterDash
import pandas as pd
import numpy as np
from plotly import graph_objects as go
import app as app_file
import setup as setup_file
import dash_core_components as dcc
import pickle
from Configuration import *

### Serve Jupyter Dash

In [ ]:
JupyterDash.infer_jupyter_proxy_config()

### Define your static parameters

In [4]:
configs = Config()
setup_applet = setup_file.create_app(configs)
setup_applet.run_server(mode='inline')

### View your sample population

In [5]:
sample_pop = configs.enc.initialize()
print(sample_pop)

[{'gene': array([14, 17, 94, 78, 27, 55, 69,  9, 68, 46, 60, 39, 81, 59, 12, 24, 97,
        73, 83, 92, 79, 64, 48, 72, 52, 26, 61, 29, 75, 93, 87, 31, 15, 47,
        20, 56, 18, 30, 50, 33, 91, 19, 90, 66, 10, 44,  3, 58,  5, 88, 51,
        42,  8, 13, 70, 62, 99, 77, 49, 32, 53, 76,  0, 23, 71, 86,  7,  4,
        36, 35, 63, 45, 89, 85, 96, 40, 28, 95, 22,  2, 16, 34,  1, 21, 84,
        11, 67, 54, 98, 65,  6, 74, 82, 57, 37, 25, 80, 43, 38, 41]), 'fitness': array([0])}
 {'gene': array([63, 84, 86, 48, 12, 96, 44, 71, 81, 67, 41, 10, 57, 31, 27, 37, 23,
        90,  9, 28, 76, 42, 99, 40, 82, 75, 36, 13, 69, 65, 25, 55, 32, 49,
        68, 56, 54, 17, 30, 19, 92, 39, 70, 78, 59,  5,  7, 50, 47, 46, 21,
        60, 18, 85,  6, 20,  4, 58, 62, 16, 72, 73, 14, 91, 89, 51, 34, 97,
        35, 33,  8, 22, 93, 11, 94, 26, 53,  3, 66,  0, 80, 98, 61, 77, 74,
        88, 38, 24, 43, 29,  2, 95, 79, 52, 83, 15, 87, 45, 64,  1]), 'fitness': array([0])}
 {'gene': array([84, 47, 59, 58, 37,

### Define your evaluation function

Your evaluation function should update all fitness values of an individual ```ind```. For example, the evaluation function for the single-objective Traveling Salesman Problem is defined below.

```
def TSP(ind):
    dist = np.loadtxt('dist.txt')

    ind['fitness'][0] = dist[0][ind['gene'][0]] 
    + np.array([dist[ind['gene'][i - 1]][ind['gene'][i]] for i in range(1, configs.gene_size)]).sum() 
    + dist[ind['gene'][-1]][0]

    return ind
```

In [6]:
def eval(pop):
    dist = np.loadtxt('dist.txt')

    for ind in pop:
        ind['fitness'][0] = dist[0][ind['gene'][0]] + np.array([dist[ind['gene'][i - 1]][ind['gene'][i]] for i in range(1, configs.gene_size)]).sum() + dist[ind['gene'][-1]][0]

    return pop

In [13]:
sample_ind = eval(sample_pop)
print(sample_ind[0])

{'gene': array([60, 93, 36,  7, 72,  1, 14, 37, 30, 15,  8, 19, 76, 88,  5, 41, 54,
       84, 28, 43, 83, 61, 40, 64, 59,  3, 89, 18, 23, 87, 66, 48, 99, 77,
       67, 13,  6, 32, 74, 75, 39, 68, 33, 10, 17, 80, 22, 51, 63, 46, 71,
       91,  0, 50, 62, 92, 34,  2, 21, 96, 69, 52, 25, 79, 53, 49, 78, 70,
       26, 57, 82,  9, 20, 85, 38, 95, 65, 12, 86, 81,  4, 11, 94, 47, 45,
       16, 73, 27, 24, 56, 31, 35, 58, 44, 97, 42, 55, 29, 90, 98]), 'fitness': array([53927])}


### Define a custom Plotly figure to display an individual's phenotypic (high-level) representation

For example, the phenotypic representation for a solution Traveling Salesman Problem is defined below as a graph network

```
def network(ind):
    locs = np.loadtxt('loc.txt')  
    
    fig = go.Figure(data=[go.Scatter(x=[locs[0][0]] + [locs[0][ind['gene'][i]] for i in range(configs.gene_size)] + [locs[0][0]],
                                     y=[locs[1][0]] + [locs[1][ind['gene'][i]] for i in range(configs.gene_size)] + [locs[1][0]],
                                     mode='lines+markers')])
    return fig
```

In [7]:
def custom(ind):
    locs = np.loadtxt('loc.txt')  
    
    fig = go.Figure(data=[go.Scatter(x=[locs[0][0]] + [locs[0][ind['gene'][i]] for i in range(configs.gene_size)] + [locs[0][0]],
                                     y=[locs[1][0]] + [locs[1][ind['gene'][i]] for i in range(configs.gene_size)] + [locs[1][0]],
                                     mode='lines+markers')])
    fig.update_layout(xaxis_title='Latitude', yaxis_title='Longitude')
    return fig

### Save your static configurations

If you are satisfied with your evaluation function and custom visualization, save it to your configurations

In [8]:
configs.eval, configs.vis = eval, custom

### Run the application

In [9]:
main_app = app_file.create_app(configs)
main_app.run_server()

Dash app running on http://127.0.0.1:8050/
